In [37]:
import pandas as pd
df = pd.read_parquet("dataset/cicdarknet.parquet")
# df.shape
df.info()
# df
df = df.drop(columns=['Label.1'], errors='ignore')
df.head()
df_tor = df[df["Label"].isin(["Tor", "Non-Tor"])]
# Convert categorical labels to numerical (Tor = 1, Non-Tor = 0)
df_tor['Label'] = df_tor['Label'].map({'Tor': 1, 'Non-Tor': 0})

print(df_tor['Label'].value_counts())  # Check class distribution
df_tor.to_parquet("Tor_Tor-Not.parquet")
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Features and target
X = df_tor.drop(columns=['Label'])
y = df_tor['Label']

# Splitting before SMOTE to avoid data leakage
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Apply SMOTE to training data only
smote = SMOTE(sampling_strategy=0.5, random_state=42)  # Adjust ratio as needed
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print("Before SMOTE:", y_train.value_counts())
print("After SMOTE:", pd.Series(y_train_resampled).value_counts())  # Class distribution after oversampling
